In [ ]:
import sagemaker
import boto3
from sagemaker.local import LocalSession
from sagemaker.model import Model
from sagemaker import get_execution_role
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import numpy as np
import json

In [ ]:
session = LocalSession()
session.config = {'local': {'local_code': True}}
role = get_execution_role()
model_url = "<model url on S3 bucket from the training step>"

role = "<IAM Role to assume>"
model_url = "<model url on S3 bucket>"
container="<ECR uri for hosting the inference endpoint"

### Create a local Endpoint with the Model Trained

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data=model_url,
                        framework_version="2.5.1",
                        role=role)


predictor = model.deploy(
        initial_instance_count=1,
        instance_type='local_gpu',
        serializer=JSONSerializer(),
        deserializer=JSONDeserializer(),
)

print(f"endpoint name: {model.endpoint_name}")

### Raw Video File

In [ ]:
from IPython.display import HTML
local_media_file = f"../test/videos/1_01_00.mp4"
HTML(f"""
    <video width="50%" alt="video" controls>
        <source src={local_media_file} type="video/mp4">
    </video>
""")

### Integrate Inference Application
With a local endpoint deployed in your environment, 
you can now create ball tracking videos by integrating the deployed endpoint with your own videos. 
A sample application is included in this repository to demonstrate the capability in action.
To run it simply do the followings in your terminal or run the below cell. If you are running this cell in SageMaker Notebook environment, you can select a tensorflow based kernels to run the notebook and skip through running ./setup.sh to install files. 

<code>
cd ../test
./setup.sh 
python application.py --input_video_file input_video_file \
--output_video_file_path output_video_file \
--endpoint_name sm_endpoint_name

In [ ]:
!cd ../test && ls && ./setup.sh && python application.py --input_video_file videos/1_01_00.mp4 --output_video_file_path videos/1_01_00.mp4 --endpoint_name local

### Prediction Results
Inference application will save the results and prediction labels (visibility) and the location of the ball in a csv file. The sample file below represents an example of the prediction results above. Take a look at the ball closely and observe the red dot following the movement of the ball.

![1_01_00_predict](https://user-images.githubusercontent.com/22420800/225199635-cfef3e45-b0e8-4eee-9218-9228b1523857.gif)

In [ ]:
### Delete Endpoint
# Delete the SageMaker endpoint and the attached resources
predictor.delete_model()
predictor.delete_endpoint()